# Import the dependent libraries

In [101]:
import panel as pn
pn.extension('plotly')
from panel.interact import interact
from panel import widgets
import plotly.express as px
import pandas as pd
import hvplot.pandas
import matplotlib.pyplot as plt
from pathlib import Path
from dotenv import load_dotenv #Just in case we need an API key.
import requests
import json
import numpy as np
%matplotlib inline
from datetime import date
from datetime import timedelta


## Fetch the data from Coinbase API
- TODO
    - Need to show data before dataframe conversion (show our process before creating function)
    - Explain how we got column names
    - 

In [102]:
def fetch_daily_data(symbol, start, end):
    pair_split = symbol.split('/') # Splitting our symbol by the '/' and creating a a list for the new values.
    symbol = pair_split[0] + '-' + pair_split[1] # symbol = BTC-USD #The API request format requires the dash.
    url = f'https://api.pro.coinbase.com/products/{symbol}/candles?start={start}&end={end}&granularity=86400'#notice the symbol insert. There are 86400 seconds in a day.
    response = requests.get(url) #getting response from website
    if response.status_code == 200: # check to make sure the response from server is good
        #if response is good then we create a dataframe by reformatting a json load.
        data = pd.DataFrame(json.loads(response.text), columns=['unix', 'low', 'high', 'open', 'close', 'volume'])
        data['date'] = pd.to_datetime(data['unix'], unit='s') # convert to a readable date
       #######

        # if we failed to get any data, print an error...otherwise write the file
        if data is None:
            print("Did not return any data from Coinbase for this symbol")
        else:
            data.to_csv(f'Coinbase_{pair_split[0] + pair_split[1]}_dailydata_{end}.csv', index=False)
    else:
        print("Did not receieve OK response from Coinbase API")

## Call the fetch function with a function focused on our three main cryptos: BTC/USD, ETH/USD, LTC/USD.

In [103]:
today = date.today()
yesterday = today - timedelta(days = 1)
yesterday = yesterday.strftime("%Y-%m-%d")
#start_date = yesterday - timedelta(days = 300)#wont let me get more then 298 days, or so.
start_date = '2015-01-01'
end_date = '2015-09-30'
#start_date = start_date.strftime("%Y-%m-%d")
#end_date = end_date.strftime("%Y-%m-%d")
cryptolist = ['BTC/USD', 'ETH/USD', 'LTC/USD']

#function to pull crypto data from coinbase api passing in crypto symbol pair, start and end date,
def fetch_main_cryptos(crypto):
    fetch_daily_data(crypto, start_date, end_date)

    
#call the function calling our API loop thrgouh crypto list and pull based on start/end date
for crypto in cryptolist:
    fetch_main_cryptos(crypto)


## Created a path to our newly created CSV files

In [104]:
BTC_path, ETH_path, LTC_path = (Path('Coinbase_BTCUSD_dailydata.csv'),
                                Path('Coinbase_ETHUSD_dailydata.csv'),
                                Path('Coinbase_LTCUSD_dailydata.csv'))
BTC_df, ETH_df, LTC_df = (pd.read_csv(BTC_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(ETH_path, index_col='date', infer_datetime_format=False, parse_dates=True),
                          pd.read_csv(LTC_path, index_col='date', infer_datetime_format=False, parse_dates=True))

## 1

In [105]:
#This function is to create our main datasets. Please edit and comment on how we should approach this.
def clean_data(df):
    df = df.dropna() # immediately drop any null values
    df = df.drop(columns=['unix']).copy() #create deep copy of df with desired columns
    df['volume_change'] = df['volume'].pct_change() #find daily percent change in volume
    df['percent_volatility'] = round(((df['high'] - df['low']) / df['high']) * 100, 2) #Finding the amount of change between the low and high, then comparing it to the high.
    df['daily_change'] = round(df['close'].pct_change(), 5) # daily pct change
    df.drop(df.head(2).index, inplace=True) # drop the unfinished and upcoming day, inclusive of NA data
    df.sort_index(inplace=True)
    return pd.DataFrame(df)
def clean_new_data(df):
    return df.dropna()
#these are the base data sets so far
BTC_df = clean_data(BTC_df)
ETH_df = clean_data(ETH_df)
LTC_df = clean_data(LTC_df)

## Current Dataframes

In [98]:
BTC_df.head(2)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2015-07-20,277.37,280.00,277.98,280.00,782.883420,-0.841636,0.94,0.00966
2015-07-21,276.85,281.27,279.96,277.32,4943.559434,0.054534,1.57,-0.00205


In [99]:
BTC_df.tail(2)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2021-01-17,33850.03,36860.0,36004.80,35820.00,19182.04935,0.154874,8.17,-0.02196
2021-01-18,34736.46,37402.0,35820.01,36624.23,16609.64108,-0.121405,7.13,0.01968


In [100]:
ETH_df.head(2)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2016-05-18,12.5,14.93,12.50,13.18,482.521826,-0.492318,16.28,-0.11544
2016-05-19,13.0,14.90,13.18,14.90,950.441205,2.739442,12.75,0.05152


In [50]:
LTC_df.head(2)

,low,high,open,close,volume,volume_change,percent_volatility,daily_change
date,,,,,,,,
2016-08-17,3.62,3.7,3.70,3.62,10.317691,9.317691,2.16,-0.02162
2016-08-18,3.62,3.7,3.62,3.70,1.000000,-0.995038,2.16,-0.07500


## Creating new data frames to compare statistical columns

In [51]:
#joined columns into new dataframe and renamed
BTC_volume = BTC_df['volume_change']
ETH_volume = ETH_df['volume_change']
LTC_volume = LTC_df['volume_change']
volume_change_df = clean_new_data(pd.concat([BTC_volume, ETH_volume, LTC_volume], axis=1))
volume_change_df.columns = ['BTC_volume_change', 'ETH_volume_change', 'LTC_volume_change']

#joined columns into new dataframe and renamed
BTC_volatility = BTC_df['percent_volatility']
ETH_volatility = ETH_df['percent_volatility']
LTC_volatility = LTC_df['percent_volatility']
volatility_df = clean_new_data(pd.concat([BTC_volatility, ETH_volatility, LTC_volatility], axis=1))
volatility_df.columns = ['BTC_volatility', 'ETH_volatility', 'LTC_volatility']


#joined columns into new dataframe and renamed
BTC_close = BTC_df['close']
ETH_close = ETH_df['close']
LTC_close = LTC_df['close']
close_df = clean_new_data(pd.concat([BTC_close, ETH_close, LTC_close], axis=1))
close_df.columns = ['BTC_close', 'ETH_close', 'LTC_close']
ETH_LTC_close_df = clean_new_data(close_df.drop(columns='BTC_close'))

#joined columns into new dataframe and renamed
BTC_daily_change = BTC_df['daily_change']
ETH_daily_change = ETH_df['daily_change']
LTC_daily_change = LTC_df['daily_change']
daily_change_df = clean_new_data(pd.concat([BTC_daily_change, ETH_daily_change, LTC_daily_change], axis=1))
daily_change_df.columns = ['BTC_daily_change', 'ETH_daily_change', 'LTC_daily_change']


## Show new Dataframes

In [52]:
volume_change_df.tail(2)

,BTC_volume_change,ETH_volume_change,LTC_volume_change
date,,,
2021-01-17,0.154874,0.025556,-0.253196
2021-01-18,-0.121405,-0.594549,-0.162030


In [53]:
volatility_df.head(1)

,BTC_volatility,ETH_volatility,LTC_volatility
date,,,
2016-08-17,2.12,5.06,2.16


In [54]:
close_df.head(1)

,BTC_close,ETH_close,LTC_close
date,,,
2016-08-17,574.45,10.82,3.62


In [55]:
ETH_LTC_close_df.head(1)

,ETH_close,LTC_close
date,,
2016-08-17,10.82,3.62


## Create functions for hvplots so they can be combined into a dashboard

In [56]:
#shows us the pct change of daily volume
def volume_change():
    return volume_change_df.hvplot(ylim=(-1, 3))
#shows us trend line of high and low volatility
def volatility():
    return volatility_df.hvplot()
#daily stock prices
def close():
    return close_df.hvplot()
#daily stock prices (ETH and LTC)
def zoom_in_close():
    return ETH_LTC_close_df.hvplot()
#daily pct change
def pct_change_plot():
    return daily_change_df.hvplot()

In [57]:
volume_change_df.hvplot(ylim=(-1, 10))

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [58]:
volatility_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [59]:
close_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [60]:
ETH_LTC_close_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

In [61]:
daily_change_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [date]   (value)

# Created hypothetical tabs for dashboard

In [62]:
tab_1 = pn.Column(
    volume_change(),
    volatility(),
    close(),
    zoom_in_close(),
    pct_change_plot()
)
dashboard = pn.Tabs(
    ('tab_1', tab_1)
)

In [63]:
dashboard.servable()

Tabs
    [0] Column
        [0] HoloViews(NdOverlay)
        [1] HoloViews(NdOverlay)
        [2] HoloViews(NdOverlay)
        [3] HoloViews(NdOverlay)
        [4] HoloViews(NdOverlay)

# Pull twitter data, then clean and create merged dataset

In [79]:
twitter_path = Path('../data/raw_data/raw_tweets_01_filter.csv')
twitter_df = pd.read_csv(twitter_path, infer_datetime_format=False, parse_dates=True)
datetime = twitter_df['time'].str.split(" ", n=1, expand = True)
twitter_df['date'] = datetime[0]
twitter_df.drop(columns='time')
twitter_df['time'] = datetime[1]
twitter_df.drop_duplicates(subset='text', inplace=True)
twitter_df.set_index('date', inplace=True)

df = pd.merge(twitter_df, volume_change_df, how='inner', left_index=True, right_index=True)
df2= pd.merge(df, volatility_df, how='inner', left_index=True, right_index=True)
df3 = pd.merge(df2, close_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df = pd.merge(df3, daily_change_df, how='inner', left_index=True, right_index=True)
twitter_analysis_df.sort_index(inplace=True)
twitter_analysis_df['category'].fillna('null', inplace=True)
#tweet_analysis_df['2020-03-30':'2021-19-01']
#df1.merge(df2, on='ID', how='left')

# Here is some example data. We can add any tables and mess with the data from here. Anything we want to add on we should do through our crypto data frames and restart kernel

In [80]:
twitter_users = twitter_analysis_df['twitter_user'].unique()
crypto_category = twitter_analysis_df['category'].unique()
crypto_category
bitcoin_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('bit')]
ethereum_tweets_df = twitter_analysis_df[twitter_analysis_df['category'].str.contains('eth')]

In [81]:
elon = twitter_users[1]

In [82]:
elon_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == elon]

In [83]:
elon_tweets_df.head(10)

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,...,LTC_volume_change,BTC_volatility,ETH_volatility,LTC_volatility,BTC_close,ETH_close,LTC_close,BTC_daily_change,ETH_daily_change,LTC_daily_change
date,,,,,,,,,,,,,,,,,,,,,
2016-08-26,officialmcafee,bitcoin,01:23:09+00:00,NaN,Cooling system for our Bitcoin mining operatio...,768982079954485248,Twitter for Android,7,10,38,...,1.899825,0.86,2.35,3.33,579.29,11.28,3.81,0.01480,0.00894,0.00794
2016-09-02,officialmcafee,bitcoin,22:38:08+00:00,NaN,Odd things like this are materializing at our ...,771839655197155328,Twitter for Android,9,38,33,...,0.571106,1.38,3.24,15.63,576.21,12.15,3.84,-0.04110,0.02273,-0.01031
2016-09-08,officialmcafee,bitcoin,21:02:16+00:00,NaN,Our first bank.of Bitcoin mining machined read...,773989858310230017,Twitter for Android,21,35,219,...,-0.795326,2.83,3.33,1.99,627.97,11.39,4.01,0.00948,-0.02148,0.01263
2016-09-09,officialmcafee,bitcoin,13:41:24+00:00,NaN,RT @officialmcafee: Our first bank.of Bitcoin ...,774241298299625472,Twitter for iPhone,0,0,0,...,19.521535,1.66,2.99,1.99,622.07,11.64,3.96,-0.00268,-0.04668,-0.00252
2016-09-15,officialmcafee,bitcoin,21:12:22+00:00,NaN,Thank you to our $MGT shareholders that called...,776529111917420544,Twitter for iPhone,3,16,26,...,0.825320,1.01,1.57,3.54,605.44,11.96,3.85,-0.00410,-0.04928,0.01050
2016-10-03,officialmcafee,bitcoin bitcoin,21:23:19+00:00,NaN,RT @tiffanymadison: Wonder what this looks lik...,783054850066149376,Twitter for iPhone,0,0,0,...,-0.659526,0.74,2.29,2.79,612.99,13.51,3.83,0.00434,0.01426,-0.00519
2016-10-03,officialmcafee,bitcoin,17:13:19+00:00,NaN,RT @MGTCI: MGT Capital Updates Shareholders on...,782991934096900096,Twitter for iPhone,0,0,0,...,-0.659526,0.74,2.29,2.79,612.99,13.51,3.83,0.00434,0.01426,-0.00519
2016-10-06,officialmcafee,generic,14:38:44+00:00,NaN,I'm keynoting the Blockchain Conference in Lon...,784040196534853632,Twitter for Android,2,5,24,...,9.484630,0.50,3.79,2.07,611.88,12.86,3.82,-0.00987,0.01260,-0.01036
2016-10-10,officialmcafee,generic,21:07:06+00:00,NaN,RT @blockchainmoney: Catch speakers like @Disr...,785587485103587328,Twitter for iPhone,0,0,0,...,-0.460132,0.60,4.60,1.30,618.72,11.78,3.79,-0.03644,0.00085,-0.01044


# Elon has limited tweets, so here is 

In [84]:
elon_tweets_df.hvplot.scatter(x='date', y='BTC_volume_change')

:Scatter   [date]   (BTC_volume_change)

In [85]:
mcafee = twitter_users[0]

In [86]:
mcafee_tweets_df = twitter_analysis_df[twitter_analysis_df['twitter_user'] == mcafee]

In [87]:

mcafee_tweets_df.head(2)

,twitter_user,category,time,sentiment,text,tweet_id,tweet_source,quote_count,reply_count,retweet_count,...,LTC_volume_change,BTC_volatility,ETH_volatility,LTC_volatility,BTC_close,ETH_close,LTC_close,BTC_daily_change,ETH_daily_change,LTC_daily_change
date,,,,,,,,,,,,,,,,,,,,,
2016-08-18,VitalikButerin,generic,14:52:20+00:00,NaN,@haq4good @AlyseKilleen The privacy technology...,766286615941484544,Twitter Web Client,0,0,1,...,-0.995038,0.69,2.71,2.16,574.09,10.77,3.70,0.00073,0.00000,-0.07500
2016-08-22,VitalikButerin,bitcoin,06:03:10+00:00,NaN,RT @el33th4xor: Which open source project gene...,767602996293218304,Twitter Web Client,0,0,0,...,-0.999889,1.32,2.65,0.00,585.20,11.10,3.99,0.00482,0.00909,0.01013


In [88]:
mcafee_tweets_df.hvplot(x='date', y='BTC_volume_change')

:Curve   [date]   (BTC_volume_change)

In [89]:
twitter_analysis_df.reset_index(inplace=True)
option_1 = twitter_analysis_df.set_index(['twitter_user', 'category', 'date'])
option_1.head(30)

time  \
twitter_user   category                                           date                         
VitalikButerin generic                                            2016-08-18  14:52:20+00:00   
               bitcoin                                            2016-08-22  06:03:10+00:00   
                                                                  2016-08-24  23:55:44+00:00   
               bitcoin bitcoin                                    2016-08-25  04:20:00+00:00   
               bitcoin bitcoin generic ethereum ethereum ether... 2016-08-26  02:09:32+00:00   
officialmcafee bitcoin                                            2016-08-26  01:23:09+00:00   
VitalikButerin null                                               2016-08-27  10:25:45+00:00   
               ethereum ethereum                                  2016-08-27  10:09:06+00:00   
                                                                  2016-08-28  09:31:48+00:00   
                                                                  2016-08-29  02:29:39+00:00   
officialmcafee bitcoin                                            2016-09-02  22:38:08+00:00   
VitalikButerin null                                               2016-09-03  05:50:01+00:00   
               ethereum ethereum                                  2016-09-04  01:35:33+00:00   
               generic                                            2016-09-04  00:20:15+00:00   
               ethereum ethereum                                  2016-09-05  22:56:42+00:00   
               ethereum                                           2016-09-05  04:22:30+00:00   
               ethereum ethereum                                  2016-09-06  13:00:43+00:00   
               generic ethereum ethereum                          2016-09-06  06:51:13+00:00   
               ethereum ethereum                                  2016-09-06  02:24:17+00:00   
joerogan       bitcoin                                            2016-09-07  19:43:13+00:00   
               generic                                            2016-09-07  05:30:28+00:00   
VitalikButerin null                                               2016-09-08  11:11:03+00:00   
               ethereum ethereum                                  2016-09-08  11:05:29+00:00   
joerogan       bitcoin bitcoin                                    2016-09-08  00:22:23+00:00   
officialmcafee bitcoin                                            2016-09-08  21:02:16+00:00   
VitalikButerin generic ethereum ethereum                          2016-09-09  08:58:35+00:00   
officialmcafee bitcoin                                            2016-09-09  13:41:24+00:00   
VitalikButerin ethereum                                           2016-09-10  11:59:03+00:00   
               bitcoin                                            2016-09-11  01:34:49+00:00   
               generic                                            2016-09-12  14:29:27+00:00   

                                                                              sentiment  \
twitter_user   category                                           date                    
VitalikButerin generic                                            2016-08-18        NaN   
               bitcoin                                            2016-08-22        NaN   
                                                                  2016-08-24        NaN   
               bitcoin bitcoin                                    2016-08-25        NaN   
               bitcoin bitcoin generic ethereum ethereum ether... 2016-08-26        NaN   
officialmcafee bitcoin                                            2016-08-26        NaN   
VitalikButerin null                                               2016-08-27        NaN   
               ethereum ethereum                                  2016-08-27        NaN   
                                                                  2016-08-28        NaN   
              

In [92]:
option_2 = twitter_analysis_df.set_index(['date', 'time'])
option_2.head(50)

twitter_user  \
date       time                             
2016-08-18 14:52:20+00:00  VitalikButerin   
2016-08-22 06:03:10+00:00  VitalikButerin   
2016-08-24 23:55:44+00:00  VitalikButerin   
2016-08-25 04:20:00+00:00  VitalikButerin   
2016-08-26 02:09:32+00:00  VitalikButerin   
           01:23:09+00:00  officialmcafee   
2016-08-27 10:25:45+00:00  VitalikButerin   
           10:09:06+00:00  VitalikButerin   
2016-08-28 09:31:48+00:00  VitalikButerin   
2016-08-29 02:29:39+00:00  VitalikButerin   
2016-09-02 22:38:08+00:00  officialmcafee   
2016-09-03 05:50:01+00:00  VitalikButerin   
2016-09-04 01:35:33+00:00  VitalikButerin   
           00:20:15+00:00  VitalikButerin   
2016-09-05 22:56:42+00:00  VitalikButerin   
           04:22:30+00:00  VitalikButerin   
2016-09-06 13:00:43+00:00  VitalikButerin   
           06:51:13+00:00  VitalikButerin   
           02:24:17+00:00  VitalikButerin   
2016-09-07 19:43:13+00:00        joerogan   
           05:30:28+00:00        joerogan   
2016-09-08 11:11:03+00:00  VitalikButerin   
           11:05:29+00:00  VitalikButerin   
           00:22:23+00:00        joerogan   
           21:02:16+00:00  officialmcafee   
2016-09-09 08:58:35+00:00  VitalikButerin   
           13:41:24+00:00  officialmcafee   
2016-09-10 11:59:03+00:00  VitalikButerin   
2016-09-11 01:34:49+00:00  VitalikButerin   
2016-09-12 14:29:27+00:00  VitalikButerin   
2016-09-15 21:12:22+00:00  officialmcafee   
2016-09-18 15:29:10+00:00  VitalikButerin   
2016-09-19 01:42:12+00:00  VitalikButerin   
2016-09-22 19:51:04+00:00  VitalikButerin   
2016-09-23 02:29:55+00:00  VitalikButerin   
2016-09-26 00:43:18+00:00  VitalikButerin   
           00:21:05+00:00  VitalikButerin   
2016-09-27 14:19:12+00:00  VitalikButerin   
2016-10-03 09:21:50+00:00  VitalikButerin   
           21:23:19+00:00  officialmcafee   
           17:13:19+00:00  officialmcafee   
2016-10-06 14:38:44+00:00  officialmcafee   
2016-10-08 11:38:08+00:00  VitalikButerin   
2016-10-10 21:07:06+00:00  officialmcafee   
           13:42:06+00:00  officialmcafee   
2016-10-12 05:32:48+00:00  VitalikButerin   
           05:27:47+00:00  VitalikButerin   
           05:25:48+00:00  VitalikButerin   
           03:23:03+00:00  VitalikButerin   
2016-10-19 03:57:29+00:00  VitalikButerin   

                                                                    category  \
date       time                                                                
2016-08-18 14:52:20+00:00                                            generic   
2016-08-22 06:03:10+00:00                                            bitcoin   
2016-08-24 23:55:44+00:00                                            bitcoin   
2016-08-25 04:20:00+00:00                                    bitcoin bitcoin   
2016-08-26 02:09:32+00:00  bitcoin bitcoin generic ethereum ethereum ethe...   
           01:23:09+00:00                                            bitcoin   
2016-08-27 10:25:45+00:00                                               null   
           10:09:06+00:00                                  ethereum ethereum   
2016-08-28 09:31:48+00:00                                  ethereum ethereum   
2016-08-29 02:29:39+00:00                                  ethereum ethereum   
2016-09-02 22:38:08+00:00                                            bitcoin   
2016-09-03 05:50:01+00:00                                               null   
2016-09-04 01:35:33+00:00                                  ethereum ethereum   
           00:20:15+00:00                                            generic   
2016-09-05 22:56:42+00:00                                  ethereum ethereum   
           04:22:30+00:00                                           ethereum   
2016-09-06 13:00:43+00:00                                  ethereum ethereum   
           06:51:13+00:00                          generic ethereum ethereum   
           02:24:17+00:00                                  ethereum ethereum   
2016-09-

In [50]:
bitcoin_tweets_df.hvplot.scatter(x='date', y='BTC_daily_change')

:Scatter   [date]   (BTC_daily_change)

In [51]:
ethereum_tweets_df.hvplot.scatter(x='date', y='ETH_daily_change')

:Scatter   [date]   (ETH_daily_change)